<a href="https://colab.research.google.com/github/ervinwilneder-leader/colab-notebooks/blob/ervinwilneder-dev/La_Granja_a_Rodar!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title ## Autenticación
#@markdown ##### Antes de comenzar, ejecutar esta celda. Google Colab solicitará permisos de acceso. 
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
from google.cloud.bigquery import magics
PROJECT_ID = 'leaderanalytics' #@param {'type': 'string'}
magics.context.project = PROJECT_ID
client = bigquery.Client(project = PROJECT_ID)

In [11]:
#@title Dependencias
%%capture
import pandas as pd
import numpy as np
import os
import time
import re
import json
import plotly.express as px

def upload_to_workspace(df, table_name, annotation="-"):
  """
  Upload a DataFrame to workspace dataset on leaderanalytics project. 
  Please be sure that your're properly logged in.
  """
  try:
    AUTH_GOOGLE_ACCOUNT = !echo $(gcloud config get-value account)

    job_config = bigquery.LoadJobConfig(
        autodetect=True, 
        write_disposition="WRITE_TRUNCATE", 
        destination_table_description=f'Uploaded from Google Colab by: {AUTH_GOOGLE_ACCOUNT[0]}. Annotation: {annotation}')

    job = client.load_table_from_dataframe(
        df, f"workspace.{table_name}_{str(round(time.time()))}", job_config=job_config
    )
    job.result()

  except Exception as e:
    print(e)

In [3]:
#@title Variables de entorno
VIDEOS = ['Ac6M1CJIBdQ','JVCRa0feGDA','RrJf7fzfubY','T6KsT9Lzez4','a04GZbB0Nd4','bgSASHMF4WE','cIvZ_balRkw','vIX8V8__-Js','477DXJB94Wo']

In [4]:
#@title Ingesta de datos
queries = dict(
  videos_basic = f"""
  SELECT *
  FROM `leaderanalytics.youtube_transfer_agg.mvw_videos_basic_LE` 
  WHERE video_id IN (
    "{'","'.join(VIDEOS)}"
    )
  """,
  videos_basic_by_country = f"""
  SELECT *
  FROM `leaderanalytics.youtube_transfer_agg.mvw_videos_basic_by_country_LE` 
  WHERE video_id IN (
    "{'","'.join(VIDEOS)}"
    )
  """,
  videos_basic_by_subscribed_status = f"""
  SELECT *
  FROM `leaderanalytics.youtube_transfer_agg.mvw_videos_basic_by_subscribed_status_LE` 
  WHERE video_id IN (
    "{'","'.join(VIDEOS)}"
    )
  """,
  videos_traffic_source_totals_timeline = f"""
  SELECT 
    REPLACE(SPLIT(_FILE_NAME, "_VIDEO_")[OFFSET(1)], ".csv", "") AS video_id,
    * EXCEPT(day),
    PARSE_DATE("%Y%m%d", CAST(day AS STRING)) AS day
  FROM `leaderanalytics.youtube.totals_timeline` 
  """,
  videos_traffic_source_top_entities_charts = f"""
  SELECT 
    REPLACE(SPLIT(_FILE_NAME, "_VIDEO_")[OFFSET(1)], ".csv", "") AS video_id,
    * EXCEPT(day),
    PARSE_DATE("%Y%m%d", CAST(day AS STRING)) AS day
  FROM `leaderanalytics.youtube.top_entities_charts`
  """,
  videos_traffic_source_top_entities_table = f"""
  SELECT  
    REPLACE(SPLIT(_FILE_NAME, "_VIDEO_")[OFFSET(1)], ".csv", "") AS video_id,
    *
  FROM `leaderanalytics.youtube.top_entities_table` 
  """,
  videos_traffic_source_totals_sums = f"""
  SELECT  
    REPLACE(SPLIT(_FILE_NAME, "_VIDEO_")[OFFSET(1)], ".csv", "") AS video_id,
    *
  FROM `leaderanalytics.youtube.totals_sums` 
  """
)

dataframes = {}

for name, sql in queries.items():
  dataframes[name] = client.query(sql).to_dataframe()

In [19]:
df = dataframes['videos_traffic_source_top_entities_charts'].query('video_id == "JVCRa0feGDA"').groupby(['day', 'traffic_source_type']).mean().reset_index()
df = df.pivot(index='day', columns='traffic_source_type', values='video_thumbnail_impressions')

fig = px.area(df, facet_col="traffic_source_type", facet_col_wrap=2, width=1000, height=1000)
fig.show()

In [22]:
df

traffic_source_type,PLAYLIST,SUBSCRIBER,YT_CHANNEL,YT_PLAYLIST_PAGE,YT_RELATED,YT_SEARCH
day,,,,,,
2022-05-03,0.0,0.0,0.0,0.0,0.0,0.0
2022-05-04,0.0,0.0,0.0,0.0,0.0,0.0
2022-05-05,0.0,0.0,0.0,0.0,0.0,0.0
2022-05-06,0.0,0.0,0.0,0.0,0.0,0.0
2022-05-07,0.0,0.0,0.0,0.0,0.0,0.0
2022-05-08,0.0,0.0,0.0,0.0,0.0,0.0
2022-05-09,0.0,0.0,0.0,0.0,0.0,0.0
2022-05-10,0.0,0.0,0.0,0.0,0.0,0.0
2022-05-11,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
with open('/content/facebook_posts.json') as f:
  fb = json.load(f)

In [24]:
records = []
for post in fb:
  data = post['video_stats']['data']

  for metric in data:
    totals = metric['totals']
    for total in totals:
      records.append({
          'id': post['id'],
          'metric': re.sub('_$', '', re.sub(r';+', '_', total['key'])).lower(),
          'value': total['value']['raw_value'],
          'start_timestamp': total['value']['date_range_start_timestamp'],
          'end_timestamp': total['value']['date_range_end_timestamp']
      })

In [25]:
df = pd.DataFrame(records)
df['start_date'] = pd.Series(df['start_timestamp'].values.astype(dtype='datetime64[s]')).dt.date
df['end_date'] = pd.Series(df['end_timestamp'].values.astype(dtype='datetime64[s]')).dt.date
df.drop(columns=['start_timestamp', 'end_timestamp'], inplace=True)

In [28]:
df

,id,metric,value,start_date,end_date
0,5137025873043935,distribution_score_count,1.133482e+00,2022-05-20,2022-06-02
1,5137025873043935,views_3s_count,0.000000e+00,2022-05-20,2022-06-02
2,5137025873043935,views_60s_es_count,0.000000e+00,2022-05-20,2022-06-02
3,5137025873043935,social_actions_count,2.600000e+02,2022-05-20,2022-06-02
4,5137025873043935,impressions_count,1.992970e+05,2022-05-20,2022-06-02
...,...,...,...,...,...
240,1093706301358454,time_watched_ms_count,1.047834e+09,2022-05-19,2022-06-02
241,1093706301358454,avg_time_watched_ms_count,1.318000e+04,2022-05-19,2022-06-02
242,1093706301358454,views_60s_es_unique,0.000000e+00,2022-05-19,2022-06-02
243,1093706301358454,negative_feedback_count_negative_feedback_type...,4.000000e+00,2022-05-19,2022-06-02


In [29]:
with open('/content/facebook_posts_lagranjaarodar_metadata.json') as f:
  fb_posts_metadata = json.load(f)

metadata_columns = ['id', 'title', 'description', 'duration_sec', 'publish_timestamp', 'thumbnail_src', 'post_type', 'post_status', 'owner_name', 'page_id_coerced']
fb_posts_metadata = pd.DataFrame(fb_posts_metadata)[metadata_columns]
fb_posts_metadata['publish_date'] = pd.Series(fb_posts_metadata['publish_timestamp'].values.astype(dtype='datetime64[s]')).dt.date
fb_posts_metadata.drop(columns=['publish_timestamp'], inplace=True)
fb_posts_metadata.query('post_type == "VIDEO"')

,id,title,description,duration_sec,thumbnail_src,post_type,post_status,owner_name,page_id_coerced,publish_date
0,320262943612011,La Granja a Rodar 🐴🐮🐓 - Apple Music,¡Los personajes de La Granja de Zenón se van a...,38.0,https://scontent.faep10-1.fna.fbcdn.net/v/t15....,VIDEO,POSTED,La Granja de Zenón,1553908531550086,2022-05-28
1,530316112059015,La Granja a Rodar 🐴🐮🐓 - Apple Music,¡Los personajes de La Granja de Zenón se van a...,38.0,https://scontent.faep10-1.fna.fbcdn.net/v/t15....,VIDEO,POSTED,El Reino Infantil,127292714017301,2022-05-26
2,429731315242948,La Granja a Rodar 🐴🐮🐓 - Vamos por el Mundo,¡Viaja alrededor del mundo en compañía de los ...,30.0,https://scontent.faep10-1.fna.fbcdn.net/v/t15....,VIDEO,POSTED,La Granja de Zenón,1553908531550086,2022-05-25
3,512591010595642,La Granja a Rodar 🐴🐮🐓 - Vamos por el Mundo,¡Viaja alrededor del mundo junto a La Granja d...,16.0,https://scontent.faep10-1.fna.fbcdn.net/v/t15....,VIDEO,POSTED,El Reino Infantil,127292714017301,2022-05-23
5,724267938725014,La Granja a Rodar 🐴🐮🐓 - Vamos por el Mundo,¡Viaja alrededor con La Granja de Zenón a Roda...,16.0,https://scontent.faep10-1.fna.fbcdn.net/v/t15....,VIDEO,POSTED,La Granja de Zenón,1553908531550086,2022-05-20
7,2842855536010729,La Granja a Rodar 🐴🐮🐓 - Vamos por el Mundo,¡Viaja alrededor del mundo en compañía de los ...,30.0,https://scontent.faep10-1.fna.fbcdn.net/v/t15....,VIDEO,POSTED,El Reino Infantil,127292714017301,2022-05-20
8,719689439224876,La Granja a Rodar 🐴🐮🐓,Se acerca una aventura muy divertida que prome...,15.0,https://scontent.faep10-1.fna.fbcdn.net/v/t15....,VIDEO,POSTED,La Granja de Zenón,1553908531550086,2022-05-19
9,1093706301358454,La Granja a Rodar 🐴🐮🐓,¿A tus peques les gustaría viajar alrededor de...,15.0,https://scontent.faep10-1.fna.fbcdn.net/v/t15....,VIDEO,POSTED,El Reino Infantil,127292714017301,2022-05-19
10,398468628857021,La Granja a Rodar 🐴🐮🐓,Se acerca una aventura muy divertida que prome...,15.0,https://scontent.faep10-1.fna.fbcdn.net/v/t15....,VIDEO,POSTED,El Reino Infantil,127292714017301,2022-05-18


In [30]:
upload_to_workspace(fb_posts_metadata, "fb_posts_metadata")

/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning:

Unable to determine type of column 'id'.



In [31]:
with open('/content/facebook_posts_lagranjaarodar_timeline.json') as f:
  fb = json.load(f)

records = []
for post in fb:
  timeline = post['data'][0]['x_axis_breakdown']

  for point in timeline:
    for metric in point:
      records.append({
          'id': post['id'],
          'metric': re.sub('_$', '', re.sub(r';+', '_', metric['key'])).lower(),
          'value': metric['value']['raw_value'],
          'start_timestamp': metric['value']['date_range_start_timestamp'],
          'end_timestamp': metric['value']['date_range_end_timestamp']
      })

fb_posts_timeline = pd.DataFrame(records)
fb_posts_timeline['start_date'] = pd.Series(fb_posts_timeline['start_timestamp'].values.astype(dtype='datetime64[s]')).dt.date
fb_posts_timeline['end_date'] = pd.Series(fb_posts_timeline['end_timestamp'].values.astype(dtype='datetime64[s]')).dt.date
fb_posts_timeline.drop(columns=['start_timestamp', 'end_timestamp'], inplace=True)
fb_posts_timeline.query('id == "429731315242948"').sort_values(by=['metric', 'start_date'])

,id,metric,value,start_date,end_date
60,429731315242948,avg_time_watched_ms_count,20097,2022-05-25,2022-05-26
65,429731315242948,avg_time_watched_ms_count,24988,2022-05-26,2022-05-27
70,429731315242948,avg_time_watched_ms_count,31023,2022-05-27,2022-05-28
75,429731315242948,avg_time_watched_ms_count,30427,2022-05-28,2022-05-29
80,429731315242948,avg_time_watched_ms_count,34242,2022-05-29,2022-05-30
85,429731315242948,avg_time_watched_ms_count,33813,2022-05-30,2022-05-31
90,429731315242948,avg_time_watched_ms_count,34889,2022-05-31,2022-06-01
95,429731315242948,avg_time_watched_ms_count,33926,2022-06-01,2022-06-02
64,429731315242948,plays_count,11354,2022-05-25,2022-05-26
69,429731315242948,plays_count,14550,2022-05-26,2022-05-27


In [32]:
upload_to_workspace(fb_posts_timeline, "fb_posts_timeline")

/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/_pandas_helpers.py:275: UserWarning:

Unable to determine type of column 'id'.

